In [1]:
!pip install wandb

import wandb
wandb.login()

wandb: Currently logged in as: rositopedro (rosito_pedro). Use `wandb login --relogin` to force relogin


True

In [2]:
! pip install datasets
! pip install evaluate
! pip install -U accelerate
! pip install -U transformers

import pandas as pd

jsons_df = pd.DataFrame()
#jsons_df = pd.DataFrame(columns=['indexglob', 'indexlocal', 'text','bookid','volume'])

data = pd.read_json("/content/drive/MyDrive/CEIA/LLM/Sarcasm_Headlines_Dataset.json", lines=True)

data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = data.drop('article_link', axis=1)

In [5]:
data.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [6]:
from sklearn.model_selection import train_test_split
df_train, df_eval = train_test_split(data, train_size=0.8, random_state=42)

In [7]:
from datasets import Dataset, DatasetDict
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "eval": Dataset.from_pandas(df_eval)
})


In [8]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
tokenized_datasets = raw_datasets.map(lambda dataset: tokenizer(dataset['headline'], truncation=True), batched=True)

Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

Map:   0%|          | 0/5342 [00:00<?, ? examples/s]

In [10]:
print(tokenized_datasets["train"][0])

{'headline': 'annoying ad turns man pro-whaling', 'is_sarcastic': 1, '__index_level_0__': 17712, 'input_ids': [101, 15703, 4748, 4332, 2158, 4013, 1011, 23687, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(["headline", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("is_sarcastic", "labels")

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=50,
    weight_decay=0.01,
    report_to="wandb",  # enable logging to W&B
    eval_strategy ='steps',
    logging_dir='./logs',
    logging_steps=50,
    metric_for_best_model = 'accuracy',
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
import torch
from sklearn.metrics import f1_score

class CustomTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        MAX: Subclassed to compute training accuracy.

        How the loss is computed by Trainer. By default, all models return the loss in
        the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)

        # MAX: Start of new stuff.
        if "labels" in inputs:
            preds = outputs.logits.detach()
            acc = (
                (preds.argmax(axis=1) == inputs["labels"])
                .type(torch.float)
                .mean()
                .item()
            )
            self.log({"accuracy": acc})
        # MAX: End of new stuff.

        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of
            # ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss

In [16]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
wandb.finish()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.592900,0.390108,0.834706,0.811365
100,0.381600,0.351094,0.853800,0.813381
150,0.296200,0.299729,0.881131,0.855056
200,0.242500,0.267336,0.893298,0.879848
250,0.279100,0.229218,0.903969,0.889891
300,0.257500,0.235830,0.904717,0.893892
350,0.241100,0.222680,0.912580,0.898677
400,0.197300,0.216434,0.911456,0.900818
450,0.235200,0.239463,0.898165,0.876082
500,0.229000,0.210712,0.914452,0.903198


eval/accuracy,▁▃▅▆▇▇██▆████
eval/f1,▁▁▄▆▇▇██▆████
eval/loss,█▇▅▃▂▂▂▂▂▁▁▁▁
eval/runtime,▂▁▁▁▁▁▁▁▂▁▁█▂
eval/samples_per_second,▇██▇████▇█▇▁▇
eval/steps_per_second,▇██▇████▇█▇▁▇
train/accuracy,▄▂▂▁▄▂▃▆▂▆▄▂▆▄▆▃▂▆▆▂▄▆▄▄█▄▄▃▄▆█▄▆█▂▄█▂▄█
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▂▅▂▃▃▂▄▁█▄▂▁▁
train/learning_rate,█▇▇▆▆▅▅▄▃▃▂▂▁


In [17]:
wandb.init()

from sklearn.metrics import classification_report

y_pred = trainer.predict(tokenized_datasets["eval"]).predictions
y_pred = np.argmax(y_pred, axis=-1)

y_true = tokenized_datasets["eval"]["labels"]
y_true = np.array(y_true)

print(classification_report(y_true, y_pred, digits=3))

wandb.finish()

              precision    recall  f1-score   support

           0      0.918     0.938     0.928      2996
           1      0.918     0.893     0.905      2346

    accuracy                          0.918      5342
   macro avg      0.918     0.915     0.916      5342
weighted avg      0.918     0.918     0.918      5342



train/accuracy,▇▅▇▇▇▅▇▇▅█▂██▇▄█▇▇█▅▂▂▅▇█▇▂▅▇▇▁███▂▇██▇█
train/epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,0.92857
train/epoch,1.0
train/global_step,668


In [18]:
wandb.init()

trainer.evaluate()

wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/accuracy,▇▅▇▇▇▅▇▇▅█▂██▇▄█▇▇█▅▂▂▅▇█▇▂▅▇▇▁███▂▇██▇█
train/epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/accuracy,0.91782
eval/f1,0.90512
